# BMI Predictor 🧮
---
## Training

In [1]:
# Libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib # save and load models

# Configuration: Random seed for reproducibility
RANDOM_STATE = 42

In [2]:
# 1. Load the cleaned BMI dataset
df_bmi = pd.read_csv('../data/bmi_cleaned.csv')

In [3]:
# 2. Define Features (X) and Target Variable (y)
X = df_bmi.drop(columns=['Index']) 
y = df_bmi['Index']

In [4]:
# 3. Split the data into Training (80%) and Testing (20%)
# We use stratify=y to ensure that the class distribution of 'Index' is the same
# in the training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

In [5]:
print(f"Size of X_train: {X_train.shape}")
print(f"Size of X_test: {X_test.shape}")

Size of X_train: (388, 4)
Size of X_test: (98, 4)


In [6]:
# (Preprocessing + Model) ⚙️

# 1. Define numeric features for scaling
# Exclude 'Gender_Encoded' as it is a binary variable.
numeric_features = ['Height', 'Weight', 'BMI_Value']

In [7]:
# 2. Create ColumnTransformer for preprocessing
# This will scale only the numeric features.
preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), numeric_features)
    ],
    remainder='passthrough' # Keep other columns (Gender_Encoded) unchanged
)

In [8]:
# 3. Create Pipeline with Preprocessor and Random Forest Classifier
# We define the Pipeline, which sequentially applies the preprocessor and then the classifier.
rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    # We set up an initial Random Forest model for the baseline
    ('classifier', RandomForestClassifier(random_state=RANDOM_STATE)) 
])

In [9]:
# Train the model 🎯

print("Starting training...")

# 1. Training: The Pipeline applies scaling (fit_transform) and then trains the model.
rf_pipeline.fit(X_train, y_train)

# 2. Prediction on the test set
y_pred_base = rf_pipeline.predict(X_test)

# 3. Initial Evaluation
print("\n--- Results ---")
print(f"Accuracy on Test: {accuracy_score(y_test, y_pred_base):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_base))

Starting training...

--- Results ---
Accuracy on Test: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00        14
           3       1.00      1.00      1.00        13
           4       1.00      1.00      1.00        26
           5       1.00      1.00      1.00        39

    accuracy                           1.00        98
   macro avg       1.00      1.00      1.00        98
weighted avg       1.00      1.00      1.00        98



In [10]:
# Asignamos el pipeline entrenado al nombre 'best_model' para seguir la convención
best_model = rf_pipeline 
final_accuracy = 1.0000 # La precisión que obtuviste

# 1. Definir el nombre del archivo de guardado
model_filename = f'best_rf_pipeline_acc{final_accuracy:.4f}.joblib'

# 2. Guardar el Pipeline completo
joblib.dump(best_model, model_filename)

print(f"\n✅ Pipeline saved: {model_filename}")
print("Training is complete and the model is ready for deployment.")


✅ Pipeline saved: best_rf_pipeline_acc1.0000.joblib
Training is complete and the model is ready for deployment.


## MLflow Experiment Tracking 🔬

In [ ]:
# Import MLflow and utilities
import sys
import os
sys.path.append(os.path.abspath('..'))

import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from scripts.utils.mlflow_utils import setup_local_mlflow, get_dataset_version, log_classification_metrics

logger.info('✅ MLflow imports ready')

In [ ]:
# Setup MLflow tracking
mlflow_dir = setup_local_mlflow()
logger.info(f'📊 MLflow tracking directory: {mlflow_dir}')

In [ ]:
# Start MLflow run with GridSearchCV
with mlflow.start_run(run_name='random-forest-training') as run:
    # Log base parameters
    mlflow.log_param('random_state', RANDOM_STATE)
    mlflow.log_param('test_size', 0.2)
    mlflow.log_param('train_size', X_train.shape[0])
    mlflow.log_param('test_size_actual', X_test.shape[0])
    
    # Get dataset version
    dataset_path = os.path.join('..', 'data', 'bmi_cleaned.csv')
    dataset_version = get_dataset_version(dataset_path)
    mlflow.log_param('dataset_version_hash', dataset_version)
    logger.info(f'Dataset version: {dataset_version}')
    
    logger.info('🔍 Starting GridSearchCV with MLflow tracking...')
    
    # Define parameter grid
    param_grid = {
        'classifier__n_estimators': [50, 100],
        'classifier__max_depth': [10, 20, None],
        'classifier__min_samples_split': [2, 5]
    }
    
    # Log parameter grid
    mlflow.log_params({
        'param_grid_n_estimators': str([50, 100]),
        'param_grid_max_depth': str([10, 20, 'None']),
        'param_grid_min_samples_split': str([2, 5]),
        'cv_folds': 5
    })
    
    # Perform GridSearchCV
    grid_search = GridSearchCV(
        rf_pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1
    )
    grid_search.fit(X_train, y_train)
    
    # Log best hyperparameters
    logger.info(f'Best parameters: {grid_search.best_params_}')
    logger.info(f'Best CV score: {grid_search.best_score_:.4f}')
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_metric('best_cv_score', grid_search.best_score_)
    
    # Use best model for final evaluation
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    
    logger.info(f'Test Accuracy: {accuracy:.4f}')
    
    # Log metrics
    mlflow.log_metric('test_accuracy', accuracy)
    
    # Create and log confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=sorted(y_test.unique()),
                yticklabels=sorted(y_test.unique()))
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix - BMI Predictor')
    
    # Save and log confusion matrix
    cm_path = '../mlruns_artifacts/confusion_matrix.png'
    os.makedirs(os.path.dirname(cm_path), exist_ok=True)
    plt.savefig(cm_path, dpi=150, bbox_inches='tight')
    plt.close()
    mlflow.log_artifact(cm_path)
    logger.info(f'✅ Confusion matrix logged')
    
    # Log model artifact
    model_path = '../mlruns_artifacts/best_model.pkl'
    import pickle
    with open(model_path, 'wb') as f:
        pickle.dump(best_model, f)
    mlflow.log_artifact(model_path)
    logger.info(f'✅ Model artifact logged')
    
    # Log model with MLflow sklearn flavor
    mlflow.sklearn.log_model(best_model, 'model')
    
    run_id = run.info.run_id
    logger.info(f'🎉 MLflow run completed! Run ID: {run_id}')

In [ ]:
# Display how to view results
from scripts.utils.mlflow_utils import get_mlflow_ui_command

print('='*60)
print('MLFLOW TRACKING COMPLETE')
print('='*60)
print(f'Run ID: {run_id}')
print(f'Experiment: bmi-predictor-random-forest')
print()
print('To view results, run:')
print(get_mlflow_ui_command())
print('='*60)